# Library

In [1]:
!conda install ../input/pyvips-install/pyvips/*.tar.bz2


######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [2]:
import os
import gc
import zipfile

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from openslide import OpenSlide
import cv2
import pyvips
import scipy.stats
import random

In [3]:
# from kaggle_datasets import KaggleDatasets
# KaggleDatasets().get_gcs_path("mayo-jpg-dataset-4x-downsampled")

# Data Load

In [4]:
train = pd.read_csv('../input/mayo-clinic-strip-ai//train.csv')
other = pd.read_csv('../input/mayo-clinic-strip-ai//other.csv')
# test = pd.read_csv('../input/mayo-jpg-dataset-4x-downsampled/test.csv')
# submission = pd.read_csv('../input/mayo-jpg-dataset-4x-downsampled/sample_submission.csv')

print(train.shape)
display(train.head())
print(other.shape)
display(other.head())
# print(test.shape)
# display(test.head())
# print(submission.shape)
# display(submission.head())

(754, 5)


,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


(396, 5)


,image_id,patient_id,image_num,other_specified,label
0,01f2b3_0,01f2b3,0,NaN,Unknown
1,01f2b3_1,01f2b3,1,NaN,Unknown
2,02ebd5_0,02ebd5,0,NaN,Unknown
3,0412ab_0,0412ab,0,NaN,Unknown
4,04414e_0,04414e,0,Hypercoagulable,Other


# Prepare dataset

Images are very large, so tile method that worked [previous competition](https://www.kaggle.com/competitions/prostate-cancer-grade-assessment/discussion/146855) would be effective.

In [5]:
def tile(img, sz=128, N=16):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    scores = []
    for im in img:
        scores.append(len(cv2.imencode(".jpg", im)[1]))
#     idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
#     img = img[idxs]
    scores, img = zip(*sorted(zip(scores, img), reverse=True, key=lambda x: x[0]))
#     high_info_ind = pd.Series(scores).where(pd.Series(scores) > 30000).idxmin()
#     print(high_info_ind is not np.nan)
    
    bg_ind = pd.Series(scores).where(pd.Series(scores) > 10000).idxmin()
    
    try:
        bg_cand = img[bg_ind:]
        for bgi, bg in enumerate(bg_cand):
            bg = bg.reshape(bg.shape[0] * bg.shape[1], bg.shape[2])
            white, _ = scipy.stats.mode(bg, axis=0)
            diff_to_white = (255,255,255) - white
            if sum(diff_to_white[0]) < 128*3:
                break
            else:
                diff_to_white = [[0,0,0]]
    except TypeError:
        print("cannot find bg")
        diff_to_white = [[0,0,0]]
        bg_cand = [np.zeros((sz,sz,3))]
        bgi = 0
    
#     if high_info_ind < N or high_info_ind is np.nan:
#         return img[:N], bg_cand[bgi], diff_to_white
#     else:
#         high_info_indexes = random.sample(list(range(high_info_ind)), N)
#         img2 = []
#         for i in high_info_indexes:
#             img2.append(img[i])
#         return img2, bg_cand[bgi], diff_to_white
#     high_info_indexes = random.sample(list(range(high_info_ind)), N)
#     img2 = []
#     for i in high_info_indexes:
#         img2.append(img[i])
    return img[:N], bg_cand[bgi], diff_to_white
    

In [6]:
# format_to_dtype = {
#    'uchar': np.uint8,
#    'char': np.int8,
#    'ushort': np.uint16,
#    'short': np.int16,
#    'uint': np.uint32,
#    'int': np.int32,
#    'float': np.float32,
#    'double': np.float64,
#    'complex': np.complex64,
#    'dpcomplex': np.complex128,
# }

# def vips2numpy(vi):
#     return np.ndarray(
#         buffer=vi.write_to_memory(),
#         dtype=format_to_dtype[vi.format],
#         shape=[vi.height, vi.width, vi.bands])
# sz=128
# N=6
# img = pyvips.Image.thumbnail(f'../input/mayo-jpg-dataset-4x-downsampled/train/006388_0.jpg', 20000)
# img = vips2numpy(img)
# shape = img.shape
# pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
# img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
# img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
# img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
# if len(img) < N:
#     img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
# scores = []
# for im in img:
#     scores.append(len(cv2.imencode(".jpg", im)[1]))
# #     idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
# #     img = img[idxs]
# scores, img = zip(*sorted(zip(scores, img), reverse=True, key=lambda x: x[0]))


In [7]:
# def vips2numpy(vi):
#     return np.ndarray(
#         buffer=vi.write_to_memory(),
#         dtype=format_to_dtype[vi.format],
#         shape=[vi.height, vi.width, vi.bands])

# format_to_dtype = {
#    'uchar': np.uint8,
#    'char': np.int8,
#    'ushort': np.uint16,
#    'short': np.int16,
#    'uint': np.uint32,
#    'int': np.int32,
#    'float': np.float32,
#    'double': np.float64,
#    'complex': np.complex64,
#    'dpcomplex': np.complex128,
# }

# image = pyvips.Image.thumbnail(f'../input/mayo-jpg-dataset-4x-downsampled/train/7b9aaa_0.jpg', 30000)
# image = vips2numpy(image)
# tile(image, sz=384, N=64)

In [8]:
format_to_dtype = {
   'uchar': np.uint8,
   'char': np.int8,
   'ushort': np.uint16,
   'short': np.int16,
   'uint': np.uint32,
   'int': np.int32,
   'float': np.float32,
   'double': np.float64,
   'complex': np.complex64,
   'dpcomplex': np.complex128,
}
def vips2numpy(vi):
    return np.ndarray(
        buffer=vi.write_to_memory(),
        dtype=format_to_dtype[vi.format],
        shape=[vi.height, vi.width, vi.bands])

# df = train

# image_dir='../input/mayo-clinic-strip-ai//train'
# image_id = df["image_id"].values[0]
# image = pyvips.Image.thumbnail(f'{image_dir}/{image_id}.tif', 500)
# image = vips2numpy(image)
# height, width, c = image.shape
# width_0, height_0 = OpenSlide(f'{image_dir}/{image_id}.tif').dimensions

# print(f"Input width: {width} height: {height}")
# print(f"org width: {width_0} height: {height_0}")

In [9]:
# from matplotlib import pyplot as plt

# image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# plt.imshow(image_gray, cmap = "gray")



# def get_cutting_point(image):
    

In [10]:
def get_cutting_point(image, width_0, height_0, box_size=20000):
    height, width, c = image.shape
    multiplier_h = height/height_0
    multiplier_w = width/width_0

    box_h = int(box_size*multiplier_h)
    box_w = int(box_size*multiplier_w)

    scores = []
    for i in range(height//int(box_h)+1):
        for j in range(width//int(box_w)+1):
            start_y = i*box_h 
            end_y = (i+1)*box_h
            start_x = j*box_w
            end_x = (j+1)*box_w

            if i == height//box_h:
                start_y = height - box_h
                end_y = height
            if j == width//box_w:
                start_x = width - box_w
                end_x = width

            if start_x < 0:
                start_x = 0
            if start_y < 0:
                start_y = 0

            scores.append((start_x, start_y, len(cv2.imencode(".jpg", image[start_y:end_y, start_x:end_x])[1])))

    x_pos , y_pos, _ = pd.DataFrame(scores, columns=["start_x", "start_y", "score"]).sort_values('score', ascending=False).iloc[0, :]
    
    return int((x_pos/width)*width_0), int((y_pos//height)*height_0)



In [11]:
# crop_size = 20000
# df = train

# output_image_dir = "/kaggle/working"
# image_dir='../input/mayo-clinic-strip-ai//train'

# for image_id in df["image_id"].values:
#     thumb = pyvips.Image.thumbnail(f'{image_dir}/{image_id}.tif', 500)
#     thumb = vips2numpy(thumb)
#     slide = OpenSlide(f'{image_dir}/{image_id}.tif')
#     height, width, c = thumb.shape
#     width_0, height_0 = slide.dimensions

#     print(f"Input width: {width} height: {height}")
#     print(f"org width: {width_0} height: {height_0}")

#     x_pos, y_pos = get_cutting_point(thumb, width_0, height_0, crop_size)
#     print(image_id, x_pos, y_pos)
#     cropped = slide.read_region((x_pos, y_pos), 0, (crop_size, crop_size))
#     cropped.convert('RGB').save(os.path.join(output_image_dir, f"{image_id}_cropped.jpg"))
    
# #     images, bg, diff_to_white = tile(cropped.convert('RGB'), sz=crop_size, N=N)

#     del cropped
#     gc.collect()

In [12]:
# cropped = slide.read_region((x_pos, y_pos), 0, (crop_size, crop_size))

In [13]:
# type(cropped)

In [14]:
def save_dataset(
    df: pd.DataFrame, 
    N=16,
    max_size=20000, 
    crop_size=384, 
    image_dir='../input/mayo-jpg-dataset-4x-downsampled/train', 
    out_dir='train_images.zip',
):
    with zipfile.ZipFile(out_dir, "w") as out_image:
        tk0 = tqdm(enumerate(df["image_id"].values), total=len(df))
        for i, image_id in tk0:
            print(f"[{i+1}/{len(df)}] image_id: {image_id}")
            thumb = pyvips.Image.thumbnail(f'{image_dir}/{image_id}.tif', 500)
            thumb = vips2numpy(thumb)
            slide = OpenSlide(f'{image_dir}/{image_id}.tif')
            height, width, c = thumb.shape
            width_0, height_0 = slide.dimensions

            print(f"Input width: {width} height: {height}")
            print(f"org width: {width_0} height: {height_0}")

            x_pos, y_pos = get_cutting_point(thumb, width_0, height_0, max_size)
            print(image_id, x_pos, y_pos)
            cropped = slide.read_region((x_pos, y_pos), 0, (max_size, max_size))
#             cropped.convert('RGB').save(os.path.join(output_image_dir, f"{image_id}_cropped.jpg"))

        #     images, bg, diff_to_white = tile(cropped.convert('RGB'), sz=crop_size, N=N)

            images, bg, diff_to_white = tile(img=cv2.cvtColor(np.array(cropped), cv2.COLOR_RGB2BGR), sz=crop_size, N=N)
            out_image.writestr(f"{image_id}_{str(diff_to_white[0][0])}_{str(diff_to_white[0][1])}_{str(diff_to_white[0][2])}_bg.jpg", cv2.imencode(".jpg", bg, [cv2.IMWRITE_JPEG_QUALITY, 100])[1])
            print(diff_to_white)
            for idx, img in enumerate(images):
#                 img = img + diff_to_white
#                 img = img / img.max()
#                 img = np.clip(img * 255, a_min = 0, a_max = 255).astype(np.uint8)
#                 img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                img = cv2.imencode(".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 100])[1]
                out_image.writestr(f"{image_id}_{idx}.jpg", img)
            del img, thumb, images, cropped, slide; gc.collect()


for i in range(1, 9):
# for i in range(8, 9):

    if i == 8:
        df = train[(i-1)*100:]
    else:
        df = train[(i-1)*100:i*100]
    
    save_dataset(
        df,
        N=16, 
        max_size=10000,
        crop_size=384, 
        image_dir='../input/mayo-clinic-strip-ai/train', 
        out_dir=f'train_images_{i}.zip'
    )

  0%|          | 0/100 [00:00<?, ?it/s]

[1/100] image_id: 006388_0
Input width: 280 height: 500
org width: 34007 height: 60797
006388_0 19918 0
[[0 0 0]]
[2/100] image_id: 008e5c_0
Input width: 100 height: 500
org width: 5946 height: 29694
008e5c_0 0 0
[[0 0 0]]
[3/100] image_id: 00c058_0
Input width: 123 height: 500
org width: 15255 height: 61801
00c058_0 0 0
[[0 0 0]]
[4/100] image_id: 01adc5_0
Input width: 500 height: 238
org width: 55831 height: 26553
01adc5_0 0 0
[[11 11 11]]
[5/100] image_id: 026c97_0
Input width: 500 height: 392
org width: 10533 height: 8267
026c97_0 547 0
[[6 0 0]]
[6/100] image_id: 028989_0
Input width: 191 height: 500
org width: 32789 height: 85695
028989_0 19913 0
[[0 5 0]]
[7/100] image_id: 029c68_0
Input width: 485 height: 500
org width: 33023 height: 34060
029c68_0 9940 0
[[ 1 39 21]]
[8/100] image_id: 032f10_0
Input width: 115 height: 500
org width: 6598 height: 28652
032f10_0 0 0
[[ 0 14  0]]
[9/100] image_id: 0372b0_0
Input width: 96 height: 500
org width: 4841 height: 25138
0372b0_0 0 0
[[ 

  0%|          | 0/100 [00:00<?, ?it/s]

[1/100] image_id: 21b106_0
Input width: 217 height: 500
org width: 13212 height: 30469
21b106_0 3226 0
[[0 0 0]]
[2/100] image_id: 2244d2_0
Input width: 317 height: 500
org width: 37330 height: 58942
2244d2_0 19783 0
[[0 0 0]]
[3/100] image_id: 2268cf_0
Input width: 165 height: 500
org width: 16413 height: 49809
2268cf_0 6465 0
cannot find bg
[[0, 0, 0]]
[4/100] image_id: 2394f6_0
Input width: 500 height: 481
org width: 36325 height: 34966
2394f6_0 0 0
[[0 0 0]]
[5/100] image_id: 23b492_0
Input width: 96 height: 500
org width: 12062 height: 62931
23b492_0 2135 0
[[0 0 0]]
[6/100] image_id: 23d2c1_0
Input width: 176 height: 500
org width: 17514 height: 49804
23d2c1_0 7562 0
[[ 0 14  0]]
[7/100] image_id: 23d2c1_1
Input width: 207 height: 500
org width: 24087 height: 58070
23d2c1_1 0 0
[[ 1 39 21]]
[8/100] image_id: 23d2c1_2
Input width: 203 height: 500
org width: 15360 height: 37825
23d2c1_2 0 0
[[0 0 0]]
[9/100] image_id: 2446b8_0
Input width: 290 height: 500
org width: 18709 height: 3

  0%|          | 0/100 [00:00<?, ?it/s]

...

[1/100] image_id: ab1b6e_0
Input width: 500 height: 351
org width: 51843 height: 36423
ab1b6e_0 0 0
[[0 0 0]]
[2/100] image_id: abc4a3_0
Input width: 207 height: 500
org width: 17545 height: 42426
abc4a3_0 7628 0
[[0 0 0]]
[3/100] image_id: abc4a3_1
Input width: 230 height: 500
org width: 24121 height: 52541
abc4a3_1 9963 0
[[0 0 0]]
[4/100] image_id: abc4a3_2
Input width: 312 height: 500
org width: 25276 height: 40548
abc4a3_2 0 0
cannot find bg
[[0, 0, 0]]
[5/100] image_id: abc682_0
Input width: 78 height: 500
org width: 7899 height: 50699
abc682_0 0 0
[[0 5 0]]
[6/100] image_id: abfcb7_0
Input width: 134 height: 500
org width: 16368 height: 60876
abfcb7_0 0 0
[[0 0 0]]
[7/100] image_id: ac2147_0
Input width: 265 height: 500
org width: 8832 height: 16640
ac2147_0 0 0
[[ 1 39 21]]
[8/100] image_id: adc6e3_0
Input width: 267 height: 500
org width: 14316 height: 26768
adc6e3_0 4343 0
[[0 0 0]]
[9/100] image_id: ae0487_0
Input width: 206 height: 500
org width: 11016 height: 26787
ae0487_

  0%|          | 0/100 [00:00<?, ?it/s]

[1/100] image_id: d33b23_0
Input width: 268 height: 500
org width: 19789 height: 36890
d33b23_0 9820 0
[[0 0 0]]
[2/100] image_id: d346ca_0
Input width: 358 height: 500
org width: 39543 height: 55239
d346ca_0 9940 0
[[0 0 0]]
[3/100] image_id: d36b5f_0
Input width: 306 height: 500
org width: 20884 height: 34107
d36b5f_0 0 0
[[ 1 39 21]]
[4/100] image_id: d380dc_0
Input width: 500 height: 104
org width: 59819 height: 12428
d380dc_0 0 0
[[0 0 0]]
[5/100] image_id: d3d21b_0
Input width: 398 height: 500
org width: 22031 height: 27657
d3d21b_0 9963 0
[[0 5 0]]
[6/100] image_id: d44b91_0
Input width: 126 height: 500
org width: 17434 height: 69169
d44b91_0 7471 0
[[0 0 0]]
[7/100] image_id: d44b91_1
Input width: 271 height: 500
org width: 27429 height: 50678
d44b91_1 0 0
[[0 0 0]]
[8/100] image_id: d4791c_0
Input width: 376 height: 500
org width: 5521 height: 7339
d4791c_0 0 0
[[ 0 14  0]]
[9/100] image_id: d4ac9a_0
Input width: 219 height: 500
org width: 15377 height: 35061
d4ac9a_0 5406 0
[

  0%|          | 0/54 [00:00<?, ?it/s]

[1/54] image_id: f322da_0
Input width: 168 height: 500
org width: 20784 height: 61781
f322da_0 9897 0
[[0 0 0]]
[2/54] image_id: f3e9f6_0
Input width: 387 height: 500
org width: 5946 height: 7676
f3e9f6_0 0 0
[[0 0 0]]
[3/54] image_id: f3e9f6_1
Input width: 500 height: 388
org width: 15496 height: 12017
f3e9f6_1 5516 0
[[0 0 0]]
[4/54] image_id: f40c69_0
Input width: 194 height: 500
org width: 17243 height: 44456
f40c69_0 7288 0
[[ 0 14  0]]
[5/54] image_id: f40c69_1
Input width: 156 height: 500
org width: 10981 height: 35086
f40c69_1 985 0
[[0 0 0]]
[6/54] image_id: f40c69_2
Input width: 236 height: 500
org width: 25227 height: 53462
f40c69_2 9941 0
[[ 0 14  0]]
[7/54] image_id: f40c69_3
Input width: 392 height: 500
org width: 25311 height: 32247
f40c69_3 9943 0
[[ 0 14  0]]
[8/54] image_id: f4287c_0
Input width: 500 height: 417
org width: 29909 height: 24958
f4287c_0 0 0
[[12 11 11]]
[9/54] image_id: f4c9ff_0
Input width: 216 height: 500
org width: 17570 height: 40589
f4c9ff_0 0 0
[[

In [15]:
!mkdir train_images

In [16]:
!mv *.zip ./train_images/

In [17]:
!tar -cvf train_images.tar train_images

train_images/
train_images/train_images_5.zip
train_images/train_images_8.zip
train_images/train_images_3.zip
train_images/train_images_1.zip
train_images/train_images_4.zip
train_images/train_images_6.zip
train_images/train_images_2.zip
train_images/train_images_7.zip
